In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

In [49]:
%load_ext autoreload
%autoreload 2
from modelling import *
from models import * 
from params import * 
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=True)
data_class.create_dataset()
modelling_data = DataModel(data = data_class,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=100,
                           custom=True)
modelling_data.create_padding()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
modifying features
['link', 'lob', 'employer', 'occupation', 'name_sex', 'firstname_lower']
['link', 'lob', 'employer', 'occupation', 'name_sex', 'firstname_lower_lstm']
**************augmenting data set ***********
modifying features
['link', 'lob', 'employer', 'occupation', 'name_sex', 'firstname_lower']
['link', 'lob', 'employer', 'occupation', 'name_sex', 'firstname_lower_lstm']
Shape of training tensor:  (20000, 100)
Shape of testing tensor:  (27, 100)


In [51]:
%load_ext autoreload 
%autoreload 2
from models import *
model_socface = Model(data = modelling_data,model_name= "knn")
model_socface.fit()
model_socface.predict()
model_socface.compute_results()
model_socface.results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ...........clf__n_neighbors=3, clf__weights=uniform; total time=   1.3s
[CV] END ...........clf__n_neighbors=3, clf__weights=uniform; total time=   1.8s
[CV] END ...........clf__n_neighbors=3, clf__weights=uniform; total time=   1.8s
[CV] END ...........clf__n_neighbors=3, clf__weights=uniform; total time=   1.7s
[CV] END ...........clf__n_neighbors=3, clf__weights=uniform; total time=   1.7s
[CV] END ..........clf__n_neighbors=3, clf__weights=distance; total time=   1.4s
[CV] END ..........clf__n_neighbors=3, clf__weights=distance; total time=   1.8s
[CV] END ..........clf__n_neighbors=3, clf__weights=distance; total time=   2.3s
[CV] END ..........clf__n_neighbors=3, clf__weights=distance; total time=   1.8s
[CV] END ..........clf__n_neighbors=3, clf__weights=distance; total time=   3.0s
[CV] END ...........clf__n_neighbors=5, cl

,accuracy_train,accuracy_test,model,custom,missing_names,acc_man,acc_women
0,0.9992,81.481481,knn,1,lstm,90.0,76.470588


In [44]:
pd.DataFrame(model_socface.model.grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_clf__binarize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.112554,0.008277,0.028362,0.001670,0.5,0.0,"{'clf__alpha': 0.5, 'clf__binarize': 0.0}",1.0,0.99975,0.99975,0.99975,0.9995,0.99975,0.000158,1
1,0.112499,0.006943,0.029233,0.001209,0.5,1.0,"{'clf__alpha': 0.5, 'clf__binarize': 1.0}",0.5,0.50000,0.50000,0.50000,0.5000,0.50000,0.000000,4
2,0.107255,0.002136,0.028944,0.002014,1.0,0.0,"{'clf__alpha': 1.0, 'clf__binarize': 0.0}",1.0,0.99975,0.99975,0.99975,0.9995,0.99975,0.000158,1
3,0.128541,0.023183,0.028711,0.002947,1.0,1.0,"{'clf__alpha': 1.0, 'clf__binarize': 1.0}",0.5,0.50000,0.50000,0.50000,0.5000,0.50000,0.000000,4
4,0.103374,0.001682,0.027320,0.002066,1.5,0.0,"{'clf__alpha': 1.5, 'clf__binarize': 0.0}",1.0,0.99975,0.99975,0.99975,0.9995,0.99975,0.000158,1
5,0.123043,0.011165,0.031081,0.003747,1.5,1.0,"{'clf__alpha': 1.5, 'clf__binarize': 1.0}",0.5,0.50000,0.50000,0.50000,0.5000,0.50000,0.000000,4


https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb

penser a fairez de la data augmentation en mixant prenom link etc pour faire plus de sample


In [ ]:
data_class.data
merged = data_class.data[data_class.features+['target']]

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame.
# First, split your DataFrame based on the 'target' value.
df_0 = merged[merged['target'] == 0]
df_1 = merged[merged['target'] == 1]

# Function to create synthetic samples
def create_synthetic_samples(data, num_samples):
    synthetic_samples = []
    for _ in range(num_samples):
        # Randomly select two samples
        samples = data.sample(n=2, replace=True, random_state=np.random.randint(0, 10000))
        # Combine their values
        synthetic_sample = {}
        for column in data.columns:
            synthetic_sample[column] = np.random.choice(samples[column].values, size=1)[0]
        synthetic_samples.append(synthetic_sample)
    return pd.DataFrame(synthetic_samples)

# Create 1000 new samples for each target
synthetic_df_0 = create_synthetic_samples(df_0, 1000)
synthetic_df_1 = create_synthetic_samples(df_1, 1000)

# Make sure the 'target' column is correctly set
synthetic_df_0['target'] = 0
synthetic_df_1['target'] = 1

# Concatenate the synthetic data with the original data to get the final augmented DataFrame
augmented_df = pd.concat([merged, synthetic_df_0, synthetic_df_1]).reset_index(drop=True)

# Now you have the original data plus 1000 synthetic samples for target 0 and 1000 for target 1
augmented_df

In [ ]:
%load_ext autoreload
%autoreload 2
from modelling import *
from models import * 
from params import * 
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=False)
data_class.create_dataset()
modelling_data = DataModel(data = data_class.data_model,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=100,
                           custom=False)
modelling_data.create_padding()
